In [1]:

import numpy as np
import pandas as pd

from sklearn.preprocessing import KBinsDiscretizer
from copy import deepcopy # Add Deepcopy for args


# pytorch
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim



import IPython
import IPython.display
import matplotlib.pyplot as plt

print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (8, 6)

1.6.0+cpu
Populating the interactive namespace from numpy and matplotlib


In [2]:
# read file
train_x = pd.read_csv('./data/train_x_df.csv')
train_y = pd.read_csv('./data/train_y_df.csv')

print("Reading Complete!")

Reading Complete!


In [3]:
train_x["is_x"] = 1
train_y["is_x"] = 0

In [10]:
train_x

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,is_x
0,0,0,0,0.993147,0.993546,0.992857,0.992966,1379.478027,3778.584961,11.240029,329.655548,903.091614,1
1,0,1,0,0.993256,0.993546,0.992712,0.992712,3438.807373,9419.426758,11.602611,1363.999268,3737.512695,1
2,0,2,0,0.992748,0.994815,0.992458,0.994815,3714.949463,10173.972656,19.579407,1222.802856,3350.688721,1
3,0,3,0,0.994779,0.995286,0.994090,0.994996,2430.264648,6666.315430,15.591008,520.159546,1426.920776,1
4,0,4,0,0.994561,0.994779,0.993727,0.994779,3062.139404,8395.172852,15.228427,2166.334473,5939.279785,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10942015,7928,1375,8,0.999708,1.000073,0.999562,0.999854,1.048120,143.449982,0.401812,0.612980,83.898582,1
10942016,7928,1376,8,0.999708,1.000000,0.999489,0.999781,0.706883,96.735909,0.409117,0.501885,68.683640,1
10942017,7928,1377,8,0.999708,1.000000,0.999489,0.999781,1.643578,224.917908,0.620982,0.788707,107.937714,1
10942018,7928,1378,8,0.999708,1.000146,0.999489,1.000000,1.204417,164.840317,0.752484,0.869763,119.042564,1


In [4]:

train_x_y = [train_x, train_y]

In [9]:
for x in train_x_y:
    print(x)
    break

          sample_id  time  coin_index      open      high       low     close  \
0                 0     0           0  0.993147  0.993546  0.992857  0.992966   
1                 0     1           0  0.993256  0.993546  0.992712  0.992712   
2                 0     2           0  0.992748  0.994815  0.992458  0.994815   
3                 0     3           0  0.994779  0.995286  0.994090  0.994996   
4                 0     4           0  0.994561  0.994779  0.993727  0.994779   
...             ...   ...         ...       ...       ...       ...       ...   
10942015       7928  1375           8  0.999708  1.000073  0.999562  0.999854   
10942016       7928  1376           8  0.999708  1.000000  0.999489  0.999781   
10942017       7928  1377           8  0.999708  1.000000  0.999489  0.999781   
10942018       7928  1378           8  0.999708  1.000146  0.999489  1.000000   
10942019       7928  1379           8  1.000000  1.000292  0.999854  1.000073   

               volume      

In [11]:

train_list = [x.set_index('sample_id') for x in train_x_y]


In [12]:
train_list

[           time  coin_index      open      high       low     close  \
 sample_id                                                             
 0             0           0  0.993147  0.993546  0.992857  0.992966   
 0             1           0  0.993256  0.993546  0.992712  0.992712   
 0             2           0  0.992748  0.994815  0.992458  0.994815   
 0             3           0  0.994779  0.995286  0.994090  0.994996   
 0             4           0  0.994561  0.994779  0.993727  0.994779   
 ...         ...         ...       ...       ...       ...       ...   
 7928       1375           8  0.999708  1.000073  0.999562  0.999854   
 7928       1376           8  0.999708  1.000000  0.999489  0.999781   
 7928       1377           8  0.999708  1.000000  0.999489  0.999781   
 7928       1378           8  0.999708  1.000146  0.999489  1.000000   
 7928       1379           8  1.000000  1.000292  0.999854  1.000073   
 
                 volume      quote_av     trades   tb_base_av 

In [13]:


train_z = pd.concat(train_list, axis=0).rename_axis('sample_id').reset_index()

In [19]:
train_z

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,is_x
0,0,0,0,0.993147,0.993546,0.992857,0.992966,1379.478027,3778.584961,11.240029,329.655548,903.091614,1
1,0,1,0,0.993256,0.993546,0.992712,0.992712,3438.807373,9419.426758,11.602611,1363.999268,3737.512695,1
2,0,2,0,0.992748,0.994815,0.992458,0.994815,3714.949463,10173.972656,19.579407,1222.802856,3350.688721,1
3,0,3,0,0.994779,0.995286,0.994090,0.994996,2430.264648,6666.315430,15.591008,520.159546,1426.920776,1
4,0,4,0,0.994561,0.994779,0.993727,0.994779,3062.139404,8395.172852,15.228427,2166.334473,5939.279785,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11893495,7928,115,8,0.995251,0.995470,0.994813,0.995397,0.477114,64.997147,0.394506,0.260363,35.472176,0
11893496,7928,116,8,0.995105,0.995397,0.994667,0.995397,1.152308,156.951431,0.533314,0.497140,67.728996,0
11893497,7928,117,8,0.995178,0.996420,0.994959,0.996128,1.609954,219.404938,0.759790,1.280783,174.547379,0
11893498,7928,118,8,0.996347,0.996347,0.995324,0.995617,0.777472,105.957024,0.379895,0.356989,48.655952,0


In [18]:
train_z[train_z['sample_id'] == 0]

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,is_x
0,0,0,0,0.993147,0.993546,0.992857,0.992966,1379.478027,3778.584961,11.240029,329.655548,903.091614,1
1,0,1,0,0.993256,0.993546,0.992712,0.992712,3438.807373,9419.426758,11.602611,1363.999268,3737.512695,1
2,0,2,0,0.992748,0.994815,0.992458,0.994815,3714.949463,10173.972656,19.579407,1222.802856,3350.688721,1
3,0,3,0,0.994779,0.995286,0.994090,0.994996,2430.264648,6666.315430,15.591008,520.159546,1426.920776,1
4,0,4,0,0.994561,0.994779,0.993727,0.994779,3062.139404,8395.172852,15.228427,2166.334473,5939.279785,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10942135,0,115,0,0.998586,0.999964,0.997498,0.998550,19381.505859,53377.167969,81.943443,8969.906250,24705.746094,0
10942136,0,116,0,0.998803,1.000254,0.997897,1.000036,11030.363281,30393.857422,65.989853,6564.782715,18088.921875,0
10942137,0,117,0,1.000072,1.001886,0.998622,1.001886,5973.658691,16485.601562,57.287891,3138.161865,8660.968750,0
10942138,0,118,0,1.002175,1.002429,0.999311,1.000689,4721.421387,13031.616211,28.643946,988.158142,2729.136719,0


In [20]:
# vwap과, open에서 vwap을 뺀 값인 diff를 계산합니다.
def make_vwap_and_diff(df):

    # 1) VAWP 계산

    # 일반적인 VWAP 공식에서 volume을 그대로 사용하지만, 여러번의 시도를 통해 tb_base_av 와 volume을 더했을 때 가장 좋은 volume이 나온다고 판단하영 사용하였습니다.
    df["volume_tb_base_av"] = df["tb_base_av"] + df["volume"]

    # open하나만을 사용하기 보다는 open(시가), high(고가), low(저가) 3개의 평균을 price로 사용하였습니다.
    df['volume_price'] = ((df['open'] + df['high'] + df['low']) / 3) * df['volume_tb_base_av']

    # price와 volume의 곱의 합을 구해줍니다.
    df['volume_price_sum'] = df.groupby(['sample_id'])['volume_price'].apply(lambda x: x.cumsum())

    # volume의 합을 구해줍니다.
    df['volume_sum'] = df.groupby(['sample_id'])['volume_tb_base_av'].apply(lambda x: x.cumsum())

    # 2 변수의 나눗셈을 통해 vwap을 계산해줍니다.
    df['vwap'] = df['volume_price_sum'] / df['volume_sum']


    # 2) diff 계산
    # 매도수익이 open을 통해 이루어진다고 알려져있기 때문에 open에서 vwap을 뺀 값을 diff로 사용했습니다.
    df["diff"] = df["open"] - df["vwap"]


    return df

In [24]:

def coindata_merger(train_x_df, train_y_df):
    
    # y dataframe time value에 1380 씩 adding
    train_y_df.time = train_y_df.time.copy() + 1380

    # x,y df merge하고 sample_id와 time 순으로 sorting
    merged_df = pd.concat([train_x_df, train_y_df])
    merged_df = merged_df.sort_values(by = ['sample_id','time']).reset_index(drop=True)

    # sample_id series orderly indexing
    sample_id_series = merged_df.sample_id.value_counts().reset_index().rename(columns = {"index" : "sample_id"})
    reset_index_series = sample_id_series.iloc[:,:1].sort_values(by = ['sample_id']).reset_index(drop=True)

    merged_df.to_hdf('./data/merged_df.h5',  key = 'merged_df')


In [25]:
coindata_merger(train_x, train_y)

In [27]:
merged_df = pd.read_hdf('./data/merged_df.h5')

In [30]:
merged_df[:1500]

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,is_x
0,0,0,0,0.993147,0.993546,0.992857,0.992966,1379.478027,3778.584961,11.240029,329.655548,903.091614,1
1,0,1,0,0.993256,0.993546,0.992712,0.992712,3438.807373,9419.426758,11.602611,1363.999268,3737.512695,1
2,0,2,0,0.992748,0.994815,0.992458,0.994815,3714.949463,10173.972656,19.579407,1222.802856,3350.688721,1
3,0,3,0,0.994779,0.995286,0.994090,0.994996,2430.264648,6666.315430,15.591008,520.159546,1426.920776,1
4,0,4,0,0.994561,0.994779,0.993727,0.994779,3062.139404,8395.172852,15.228427,2166.334473,5939.279785,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0,2875,0,0.998586,0.999964,0.997498,0.998550,19381.505859,53377.167969,81.943443,8969.906250,24705.746094,0
1496,0,2876,0,0.998803,1.000254,0.997897,1.000036,11030.363281,30393.857422,65.989853,6564.782715,18088.921875,0
1497,0,2877,0,1.000072,1.001886,0.998622,1.001886,5973.658691,16485.601562,57.287891,3138.161865,8660.968750,0
1498,0,2878,0,1.002175,1.002429,0.999311,1.000689,4721.421387,13031.616211,28.643946,988.158142,2729.136719,0


In [21]:
train_z = make_vwap_and_diff(train_z)